In [88]:
# Loading crime csv to RDD
# Replace with your directory
crime_csv = sc.textFile('file:/Users/zhuorulin/Documents/DataScience/datasets/crime.csv',use_unicode=False)

In [82]:
# Convert csv to DataFrame
from csv import reader # Warning: csv.reader does not support unicode decode
from pyspark.sql import SQLContext
from collections import defaultdict
import datetime

In [99]:
# Use csv.reader to read raw binary
lines_rdd = crime_csv.mapPartitions(reader)\
.map(lambda line: [x.decode('utf-8') for x in line] )# Decode with utf-8 codec
# store columns values. Also a search table for searching column name using index
schemas = lines_rdd.take(1)[0]
# filter out first row
lines = lines_rdd.filter(lambda x: x!=schemas)

In [104]:
# This dict makes it easier to search for column index using column name
colname2idx = defaultdict()
for idx,colname in enumerate(schemas):
    colname2idx[colname] = idx
# Example
print(colname2idx['Agency'])

3


# Null Checking

In [107]:
def checkNull(string):
    #Assume Unicode String
    # Step 1: Check for length 0 i.e '' field
    if len(string)==0:
        return 'INVALID'
    # Step 2: Check for 'nan'
    elif string=='nan':
        return 'INVALID'
    else:
        return 'VALID'

In [115]:
# Example: Check row
NULL_TABLE = lines.map(lambda line:[checkNull(x) for x in line])
print(NULL_TABLE.take(1))

[['VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'INVALID', 'INVALID', 'VALID', 'VALID', 'INVALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'VALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID', 'VALID', 'VALID', 'VALID']]


# Date Cleaning Method
Format Reference: http://strftime.org/

In [102]:
date_format = '%m/%d/%Y %I:%M:%S %p'
# If date is in wrong format it would not be sucessfully converted to datetime object.
def checkDate(line,date_format):
    try:
        datetime.datetime.strptime(line,date_format)
        return 'VALID'
    except:
        return 'INVALID'

########################################
# Example
Created_Date = lines.map(lambda x: x[colname2idx['Created Date']])
# Change encoder to utf-8
# Created_Date = Created_Date.map(lambda x: x).map(lambda x: x.encode('latin-1').strip())
Created_Date_validity = Created_Date.map(lambda x: checkDate(x,date_format))

In [106]:
#Created_Date_validity.countByValue()
# Should return defaultdict(int, {'VALID': 1413705})

# Data Frame Actions

In [105]:
# Converted to DataFrame
df = lines.toDF(schemas)
# Print out schemas to remind filed names
#df.printSchema()

In [27]:
# Create sql context to interact easier
sqlContext = SQLContext(sc)
df.registerTempTable('crime')
# USE sqlContext.sql(<sql code>) to interact with dataframe

In [9]:
# Or using data frame method
created_date = df.select('Created Date')